<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/cost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loss Function

*Author: Alexander Del Toro Barba*

## Overview

**Loss Minimization**

$\min _{W}\left\{L(W):=\frac{1}{m} \sum_{i=1}^{m} \ell\left(W ; x_{i}, y_{i}\right)+\lambda r(W)\right\}$

## Mean Absolute Error

* Computes the mean of absolute difference between labels and predictions.


## Kullback-Leibler divergence loss 

* xxx


## Sparse Categorical Crossentropy

* xxx

# RNN Model

## Import & Prepare Data

In [0]:
import tensorflow as tf
import datetime, os

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4423680/4422102 [==============================] - 0s 0us/step


## Choose Loss Function

In [0]:
loss = 'sparse_categorical_crossentropy'
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# loss = 'mae'

# https://www.tensorflow.org/api_docs/python/tf/keras/losses/

## Define Model & Run

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=5, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 79us/sample - loss: 1.7161 - acc: 0.7491 - val_loss: 1.6966 - val_acc: 0.7651
Epoch 2/5
60000/60000 [==============================] - 5s 77us/sample - loss: 1.6837 - acc: 0.7775 - val_loss: 1.6932 - val_acc: 0.7672
Epoch 3/5
60000/60000 [==============================] - 5s 77us/sample - loss: 1.6513 - acc: 0.8096 - val_loss: 1.6577 - val_acc: 0.8022
Epoch 4/5
60000/60000 [==============================] - 5s 75us/sample - loss: 1.6193 - acc: 0.8417 - val_loss: 1.6346 - val_acc: 0.8268
Epoch 5/5
60000/60000 [==============================] - 4s 74us/sample - loss: 1.6146 - acc: 0.8461 - val_loss: 1.6280 - val_acc: 0.8326
